# 获取视频列表

In [1]:
import os
import re
import requests

In [2]:
def get_signature(url):
    sign = os.popen('node xigua.js {url}'.format(url='"'+url+'"')).read()
    return sign

In [3]:
def get_video_list(max_time=0):
    main_url = 'https://www.ixigua.com/home/75436727443/'
   # 初始化 session
    sess = requests.Session()
    sess.headers.update({
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
        'referer': 'https://www.ixigua.com/home/75436727443/',
    })

    # 获取 cookies
    sess.get('https://i.snssdk.com/slardar/sdk.js?bid=xigua_video_web_pc')
    data = '{"region":"cn","aid":1768,"needFid":false,"service":"www.ixigua.com","migrate_info":{"ticket":"","source":"node"},"cbUrlProtocol":"https","union":true}'
    sess.post('https://ttwid.bytedance.com/ttwid/union/register/', data=data)
    
    # 从 html 中获取 token ，并更新 headers
    # 若提示 referer异常，可重新初始化 session，然后导入上一步获取的 cookies
    # 或索性将 tt-anti-token 置为空字符串
    resp = sess.get(main_url)
    try:
        token = re.search('"csrfToken":"(.*?)","isLogin"', resp.text).group(1)
    except:
        token = ''
    sess.headers.update({'tt-anti-token': token})

    # 获取视频信息
    params = {
        'author_id': '75436727443',
        'type': 'video',
        'max_time': max_time,
        '_signature': get_signature("/api/videov2/author/video")
    }

    resp = sess.get('https://www.ixigua.com/api/videov2/author/video', params=params)
    
    # 返回解析后的数据
    resp_json = resp.json()
    if resp_json['data']['message'] == "success":
        return resp_json['data']['data']
    else:
        print('视频获取失败。')
        return None

In [4]:
all_video_urls = []
max_time = 0

while True:
    data = get_video_list(max_time=max_time)
    if data:
        print(max_time, len(data))
        try:
            video_urls = ['https://www.ixigua.com'+i['article_url'].split('group')[1] for i in data]
        except:
            video_urls = [i['article_url'] for i in data]
        all_video_urls.extend(video_urls)
        # 更新 max_time
        max_time = data[-1]['publish_time']
    # 若 data 为空列表，退出死循环
    else:
        break

0 30
1609312789 30
1608687239 30
1608020742 30
1607401931 30
1606785031 30
1606119137 30
1605488994 30
1604821439 30
1604191656 30
1603522006 30
1602895573 30
1602132010 30
1601339840 30
1600489597 30
1599628411 30
1598781173 30
1597968479 30
1597116763 30
1596257086 30
1595392469 30
1594525853 30
1593508467 30
1592269890 30
1590986926 30
1589693451 30
1587971543 30
1585755369 30
1584607859 19


In [5]:
len(all_video_urls)

859

# 批量下载视频

In [6]:
import os
from tqdm.notebook import tqdm

def download_from_url(url, save_path):
    response = requests.get(url, stream=True) 
    file_size = int(response.headers['content-length']) 
    if os.path.exists(save_path):
        first_byte = os.path.getsize(save_path) 
    else:
        first_byte = 0
    if first_byte >= file_size: 
        return file_size
    header = {"Range": f"bytes={first_byte}-{file_size}"} 
    pbar = tqdm(
        total=file_size, initial=first_byte,
        unit='B', unit_scale=True, desc=save_path)
    req = requests.get(url, headers=header, stream=True) 
    with(open(save_path, 'ab')) as f:
        for chunk in req.iter_content(chunk_size=1024): 
            if chunk:
                f.write(chunk)
                pbar.update(1024)
    pbar.close()
    return file_size

In [ ]:
for url in all_video_urls:
    params = {
        'url': url, 
        'select': 'ixigua'
    }

    r = requests.get('https://www.videofk.com/ixigua-video-download/search', params=params)

    # 获取视频地址
    video_url = re.search('data-video="(.*?)"', r.text).group(1)
    video_url = video_url.replace('amp;', '')

    # 获取视频题目
    video_title = re.search('data-title="(.*?)"/>\n', r.text).group(1)
    print(f'开始下载「{video_title}」......')
    download_from_url(video_url, './videos/'+video_title+'.mp4')